In [140]:
! pip install pandas

In [141]:
# imports

import pandas as pd
import numpy as np
import os


# Get the current directory
current_dir = os.getcwd()

In [142]:
test = True

In [143]:
# load in the world new data

current_dir = os.getcwd()

if test:
    print('current_dir: ' + str(current_dir) + '\n')

#csv_path = current_dir + "\Top_25_World_News_2018_2023\WorldNewsData.csv"
csv_path = r"{}\Top_25_World_News_2018_2023\WorldNewsData.csv".format(current_dir)

if test:
    print('csv_path: ' + str(csv_path) + '\n')

# convert csv to pandas dataframe
news_data = pd.read_csv(csv_path)

if test:
    print('len(news_data)    :' + str(len(news_data))    + '\n')
    print('news_data.iloc[0] :' + str(news_data.iloc[0]) + '\n')
    print('news_data.head()  :' + str(news_data.head())  + '\n')

current_dir: c:\Users\vanvi\Desktop\workspace\cmpe-256-project

csv_path: c:\Users\vanvi\Desktop\workspace\cmpe-256-project\Top_25_World_News_2018_2023\WorldNewsData.csv

len(news_data)    :1859

news_data.iloc[0] :Date                                          May 01, 2018
Top1     North Korea to open its sky, South Korean medi...
Top2           The Mueller probe ain't ending anytime soon
Top3     BRAND NEW: 2018 – Renault Alpine A110 – Start-...
Top4     The story of the Midwestern innocent let loose...
Top5     Mattis tempers optimistic outlook on Afghanist...
Top6     Facebook is taking on Tinder with new dating f...
Top7     OECD chief says Greece deserves debt relief af...
Top8                     Facebook to launch dating service
Top9     U.N. could help Myanmar gather evidence of cri...
Top10    PTSD veterans 'cured' with ecstasy treatment i...
Top11    May Day protesters demand better rights for wo...
Top12    China’s Social Credit System Punishes Low-Scor...
Top13             

In [144]:
# load in the stock data

current_dir = os.getcwd()
dirs = [
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "forbes2000"),
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "nasdaq"),
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "nyse"),
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "sp500"),
       ]

#csv_files = [os.path.join(csv_dir, file) for file in os.listdir(csv_dir)]

if test:
    print('current_dir: ' + str(current_dir) + '\n')
    #print('csv_files  : ' + str(csv_files) + '\n')

# save list to handle duplicate stocks
processed_files = set()
stock_data      = []

# convert csv to pandas dataframe then combine
for dir in dirs:
    csv = [os.path.join(dir, file) for file in os.listdir(dir)]
    for file in csv:
        # take only first part of file name (leave out csv)
        file_name = os.path.splitext(os.path.basename(file))[0]
        # if duplicate, continue and skip
        if file_name in processed_files:
            continue
        # read in file and append
        df = pd.read_csv(file)
        df['Stocks'] = file_name
        stock_data.append(df)
        processed_files.add(file_name)

stock_data = pd.concat(stock_data, ignore_index=True)

if test:
    print('len(stock_data)    :' + str(len(stock_data))    + '\n')
    print('stock_data.iloc[0] :' + str(stock_data.iloc[0]) + '\n')
    print('stock_data.iloc[1] :' + str(stock_data.iloc[1]) + '\n')
    print('stock_data.head()  :' + str(stock_data.head()) + '\n')

current_dir: c:\Users\vanvi\Desktop\workspace\cmpe-256-project

len(stock_data)    :18988384

stock_data.iloc[0] :Date              18-11-1999
Low                28.612303
Open               32.546494
Volume            62546380.0
High               35.765381
Close              31.473534
Adjusted Close     27.066582
Stocks                     A
Name: 0, dtype: object

stock_data.iloc[1] :Date              19-11-1999
Low                28.478184
Open               30.713518
Volume            15234146.0
High               30.758226
Close              28.880545
Adjusted Close     24.836662
Stocks                     A
Name: 1, dtype: object

stock_data.head()  :         Date        Low       Open      Volume       High      Close  \
0  18-11-1999  28.612303  32.546494  62546380.0  35.765381  31.473534   
1  19-11-1999  28.478184  30.713518  15234146.0  30.758226  28.880545   
2  22-11-1999  28.657009  29.551144   6577870.0  31.473534  31.473534   
3  23-11-1999  28.612303  30.400572   5975

In [145]:
# remove dates older than May 01, 2018

# stock_data.iloc[1] :Date              19-11-1999
stock_data['Date'] = pd.to_datetime(stock_data['Date'], format='%d-%m-%Y')
filt_new_data = stock_data[stock_data['Date'] >= '2018-05-01']
filt_new_data = filt_new_data.reset_index(drop=True)

if test:
    print('len(filt_new_data)    :' + str(len(filt_new_data))    + '\n')
    print('filt_new_data.head()  :' + str(filt_new_data.head())  + '\n')

len(filt_new_data)    :3930369

filt_new_data.head()  :        Date        Low       Open     Volume       High      Close  \
0 2018-05-01  65.489998  65.629997  3135100.0  66.349998  66.239998   
1 2018-05-02  65.769997  66.000000  2240500.0  66.860001  65.910004   
2 2018-05-03  64.860001  65.769997  2365900.0  66.459999  66.339996   
3 2018-05-04  65.540001  66.029999  1330800.0  67.250000  67.000000   
4 2018-05-07  67.070000  67.160004  1468700.0  67.980003  67.389999   

   Adjusted Close Stocks  
0       64.348511      A  
1       64.027931      A  
2       64.445656      A  
3       65.086807      A  
4       65.465668      A  



In [ ]:
# convert news data format to stocks data

#news_data.iloc[0] :Date                                          May 01, 2018
news_data2 = np.deepcopy(news_data)
news_data['Date'] = pd.to_datetime(news_data['Date'], infer_datetime_format=True, errors='coerce')

if test:
    print('len(news_data2)      :' + str(len(news_data2))      + '\n')
    print('news_data2.iloc[50] :' + str(news_data2.iloc[50])   + '\n')
    print('news_data2.head()    :' + str(news_data2.head())    + '\n')


C:\Users\vanvi\AppData\Local\Temp\ipykernel_18620\1290378232.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  news_data2['Date'] = pd.to_datetime(news_data['Date'], infer_datetime_format=True, errors='coerce')


NameError: name 'news_data2' is not defined

In [120]:
# Merge both data sets


# No dates for stocks, means news go to next available data (forward setting)
news_data['Date']  = pd.to_datetime(news_data['Date'], dayfirst=True)
stock_data['Date'] = pd.to_datetime(stock_data['Date'], dayfirst=True)

# Merge news_data with stock_data, filling the missing dates
combined_data = pd.merge_asof(news_data.sort_values(by='Date'), 
                              stock_data.sort_values(by='Date'), 
                              on='Date', 
                              direction='forward')

if test:
    print('len(combined_data)      :' + str(len(combined_data))     + '\n')
    print('combined_data.iloc[50]  :' + str(combined_data.iloc[50]) + '\n')
    print('combined_data.head()    :' + str(combined_data.head()  ) + '\n')

C:\Users\vanvi\AppData\Local\Temp\ipykernel_18620\1036640602.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  news_data['Date']  = pd.to_datetime(news_data['Date'], dayfirst=True)


ValueError: Merge keys contain null values on right side

In [75]:
if test:
    print('(combined_data)      :' + str((combined_data))     + '\n')

(combined_data)      :           Date                                               Top1  \
0    2018-05-01  North Korea to open its sky, South Korean medi...   
1    2018-05-02  'What have they got to fear? ' Trump threatens...   
2    2018-05-03  Hollywood isn’t simply churning out crummy rem...   
3    2018-05-04  Donald Trump says London hospital is 'like a w...   
4    2018-05-05  Missing teen’s mom implores Canada to take not...   
...         ...                                                ...   
1854 2023-04-26  Uyghur death in Thai refugee detention center ...   
1855 2023-04-27  Woman whose accusation led to the lynching of ...   
1856 2023-04-28  An Ominous Heating Event Is Unfolding in the O...   
1857 2023-04-29  Russian Occupied Ukrainian City Under 'Intense...   
1858 2023-04-30  Israel will raze Khan al-Ahmar, no timetable f...   

                                                   Top2  \
0           The Mueller probe ain't ending anytime soon   
1     Best Garage H